In [1]:
import numpy as np 
import matplotlib.pyplot as plt
import glob
import pandas as pd
import cv2

from keras.models import Model, Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization, Dropout
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, Flatten, MaxPooling2D, Dense, Input, Reshape, Concatenate, GlobalAveragePooling2D, BatchNormalization, Dropout, Activation, GlobalMaxPooling2D
from tensorflow.keras.utils import Sequence
import os
import seaborn as sns
from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications import ResNet101
from tensorflow.keras.applications import ResNet152
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications import Xception
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.applications import DenseNet169
from tensorflow.keras.applications import NASNetMobile
from tensorflow.keras.applications import EfficientNetB7

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn import svm
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier

import warnings
from sklearn import preprocessing

from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import matthews_corrcoef

In [2]:
warnings.filterwarnings('ignore')

%load_ext autotime
RANDOM_STATE = 42

In [3]:
#arithmetic
#blur
#color
#contrast
#convolutional
#hybrid
#random
#Dataset
print(os.listdir(r"D:\Cancer\aug_dataset\hybrid"))
SIZE = 224

['testing', 'training']
time: 104 ms


In [4]:
#Capture training data and labels into respective lists
train_images = []
train_labels = [] 

for directory_path in glob.glob(r"D:\Cancer\aug_dataset\hybrid\training\*"):
    label = directory_path.split("\\")[-1]
    print(label)
    for img_path in glob.glob(os.path.join(directory_path, "*.jpg")):
        #print(img_path)
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)       
        img = cv2.resize(img, (SIZE, SIZE))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        train_images.append(img)
        train_labels.append(label)

#Convert lists to arrays        
train_images = np.array(train_images)
train_labels = np.array(train_labels)

Non-Tumor
Non-Viable-Tumor
Viable
time: 1min 33s


In [5]:
train_images.shape

(3660, 224, 224, 3)

time: 5 ms


In [6]:
test_images = []
test_labels = [] 
for directory_path in glob.glob(r"D:\Cancer\aug_dataset\hybrid\testing\*"):
    fruit_label = directory_path.split("\\")[-1]
    for img_path in glob.glob(os.path.join(directory_path, "*.jpg")):
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        img = cv2.resize(img, (SIZE, SIZE))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        test_images.append(img)
        test_labels.append(fruit_label)

#Convert lists to arrays                
test_images = np.array(test_images)
test_labels = np.array(test_labels)

time: 21.7 s


In [7]:
test_images.shape

(916, 224, 224, 3)

time: 2.93 ms


In [8]:
#Encode labels from text to integers.

le = preprocessing.LabelEncoder()
le.fit(test_labels)
test_labels_encoded = le.transform(test_labels)
le.fit(train_labels)
train_labels_encoded = le.transform(train_labels)

x_train, y_train, x_test, y_test = train_images, train_labels_encoded, test_images, test_labels_encoded

time: 6.22 ms


In [9]:
x_train, x_test = x_train / 255.0, x_test / 255.0

#One hot encode y values for neural network. 
from tensorflow.keras.utils import to_categorical
y_train_one_hot = to_categorical(y_train)
y_test_one_hot = to_categorical(y_test)

time: 5.69 s


In [10]:
#PRE_model = Xception(weights='imagenet', include_top=False, input_shape=(SIZE, SIZE, 3))
#PRE_model = VGG16(weights='imagenet', include_top=False, input_shape=(SIZE, SIZE, 3))
#PRE_model = ResNet50(weights='imagenet', include_top=False, input_shape=(SIZE, SIZE, 3))
#PRE_model = VGG19(weights='imagenet', include_top=False, input_shape=(SIZE, SIZE, 3))
#PRE_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(SIZE, SIZE, 3))
PRE_model = NASNetMobile(weights='imagenet', include_top=False, input_shape=(SIZE, SIZE, 3))

time: 7.73 s


In [11]:
for layer in PRE_model.layers:
    layer.trainable = False

time: 19.7 ms


In [12]:
top_model = PRE_model.output
top_model = Flatten(name="flatten")(top_model)
top_model = Dense(4096, activation='relu')(top_model)
top_model = Dense(1072, activation='relu')(top_model)
top_model = Dropout(0.2)(top_model)
model = Model(inputs=PRE_model.input, outputs=top_model)

time: 4.78 s


In [13]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 stem_conv1 (Conv2D)            (None, 111, 111, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 stem_bn1 (BatchNormalization)  (None, 111, 111, 32  128         ['stem_conv1[0][0]']             
                                )                                                             

 normal_bn_1_7 (BatchNormalizat  (None, 14, 14, 88)  352         ['normal_conv_1_7[0][0]']        
 ion)                                                                                             
                                                                                                  
 activation_106 (Activation)    (None, 14, 14, 88)   0           ['normal_bn_1_7[0][0]']          
                                                                                                  
 activation_108 (Activation)    (None, 14, 14, 88)   0           ['adjust_bn_7[0][0]']            
                                                                                                  
 activation_110 (Activation)    (None, 14, 14, 88)   0           ['adjust_bn_7[0][0]']            
                                                                                                  
 activation_112 (Activation)    (None, 14, 14, 88)   0           ['adjust_bn_7[0][0]']            
          

In [14]:
feature_extractor=model.predict(x_train)

features = feature_extractor.reshape(feature_extractor.shape[0], -1)

X_for_ML = features

time: 3min 26s


In [15]:
ML_model = MLPClassifier(random_state=1, max_iter=100)
# Train the model on training data
ML_model.fit(X_for_ML, y_train) 

MLPClassifier(max_iter=100, random_state=1)

time: 3.66 s


In [16]:
X_test_feature = model.predict(x_test)
X_test_features = X_test_feature.reshape(X_test_feature.shape[0], -1)

time: 42.4 s


In [17]:
prediction_ML = ML_model.predict(X_test_features)
#Inverse le transform to get original label back. 
prediction_ML = le.inverse_transform(prediction_ML)

time: 13.8 ms


In [18]:
cnf_matrix = confusion_matrix(test_labels, prediction_ML)

FP = cnf_matrix.sum(axis=0) - np.diag(cnf_matrix) 
FN = cnf_matrix.sum(axis=1) - np.diag(cnf_matrix)
TP = np.diag(cnf_matrix)
TN = cnf_matrix.sum() - (FP + FN + TP)
FP = FP.astype(float)
FN = FN.astype(float)
TP = TP.astype(float)
TN = TN.astype(float)
# Sensitivity, hit rate, recall, or true positive rate
TPR = TP/(TP+FN)
# Specificity or true negative rate
TNR = TN/(TN+FP) 

FPR = FP/(TN+FP) 

label_encoder = LabelEncoder()
prediction_auc = label_encoder.fit_transform(prediction_ML)
test_auc = label_encoder.fit_transform(test_labels)
fpr, tpr, thresholds = metrics.roc_curve(test_auc, prediction_auc, pos_label=2)
auc = metrics.auc(fpr, tpr)

time: 29.8 ms


In [19]:
print("ACC  = ", float("{0:.2f}".format(accuracy_score(test_labels, prediction_ML))))
print ("AUC = ",float("{0:.3f}".format(auc)))
print("MCC = ", float("{0:.3f}".format(matthews_corrcoef(test_labels, prediction_ML))))
print("Specificity  = ", float("{0:.2f}".format(sum(TNR)/3)))
print("Sensitivity  = ", float("{0:.2f}".format(sum(TPR)/3)))

ACC  =  0.94
AUC =  0.95
MCC =  0.899
Specificity  =  0.97
Sensitivity  =  0.93
time: 17.4 ms


In [20]:
import math
acc = (TP+TN)/(TP+TN+FN+FP)
div = (TP+FP)*(TP+FN)*(TN+FP)*(TN+FN)
up = (TP*TN)-(FP*FN)
mcc0 = up[0]/math.sqrt(div[0])
mcc1 = up[1]/math.sqrt(div[1])
mcc2 = up[2]/math.sqrt(div[2])
class_name = [0,1,2]

print("accuracy Each Class: ", acc)
print("auc for each class : ")
for p in class_name:
    fprEC, tprEC, thresholds = metrics.roc_curve(test_auc, prediction_auc, pos_label=p)
    aucEC = metrics.auc(fprEC, tprEC)
    print(aucEC)
    
print("MCC Each Class: ", mcc0, mcc1, mcc2)
print("Specificity Each Class: ", TNR)
print("Sensitivity Each Class: ", TPR)

accuracy Each Class:  [0.95742358 0.95633188 0.95742358]
auc for each class : 
0.038235716336491504
0.6152940779711319
0.9495357789855071
MCC Each Class:  0.9147152390330765 0.8800469467078044 0.8980345572980499
Specificity Each Class:  [0.9526749  0.96175637 0.9859375 ]
Sensitivity Each Class:  [0.9627907  0.93809524 0.89130435]
time: 9.95 ms
